# KNN

In [13]:
import pandas as pd
import numpy as np  
from sklearn.preprocessing import LabelEncoder

In [5]:
df = {'Brightness':[40,50,60,10,70,60,25],
      'Saturation':[20,50,90,25,70,10,80],
      'Class':['Red','Blue','Blue','Red','Blue','Red','Blue']}
df = pd.DataFrame(df)
df

,Brightness,Saturation,Class
0,40,20,Red
1,50,50,Blue
2,60,90,Blue
3,10,25,Red
4,70,70,Blue
5,60,10,Red
6,25,80,Blue


In [16]:
le = LabelEncoder()
y = le.fit_transform(y)
y
pred_X = [20,35]

In [12]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

In [25]:
distances = []
for pts in df.iloc[:, :].values:
    print(pts)
    distances.append((euclidean_distance(pts[:-1], pred_X),pts[-1]))
print(distances)



[40 20 'Red']
[50 50 'Blue']
[60 90 'Blue']
[10 25 'Red']
[70 70 'Blue']
[60 10 'Red']
[25 80 'Blue']
[(np.float64(25.0), 'Red'), (np.float64(33.54101966249684), 'Blue'), (np.float64(68.00735254367721), 'Blue'), (np.float64(14.142135623730951), 'Red'), (np.float64(61.032778078668514), 'Blue'), (np.float64(47.16990566028302), 'Red'), (np.float64(45.27692569068709), 'Blue')]


In [26]:
distances.sort(key=lambda x: x[0])

In [27]:
distances

[(np.float64(14.142135623730951), 'Red'),
 (np.float64(25.0), 'Red'),
 (np.float64(33.54101966249684), 'Blue'),
 (np.float64(45.27692569068709), 'Blue'),
 (np.float64(47.16990566028302), 'Red'),
 (np.float64(61.032778078668514), 'Blue'),
 (np.float64(68.00735254367721), 'Blue')]

In [28]:
k = 5
count = {}
for i in range(k):
    if distances[i][1] in count:
        count[distances[i][1]] += 1
    else:
        count[distances[i][1]] = 1

In [29]:
count

{'Red': 3, 'Blue': 2}

In [30]:
def PrintMaxClass(count):
    max_count = 0
    max_class = None
    for key in count:
        if count[key] > max_count:
            max_count = count[key]
            max_class = key
    return max_class

prediction = PrintMaxClass(count)
print(prediction)

Red


# Naive Bayes classification

$$P(A|B) = \frac{P(B|A).P(A)}{P(B)}$$

In [33]:
import pandas as pd

url = 'https://raw.githubusercontent.com/sjwhitworth/golearn/master/examples/datasets/tennis.csv'

df = pd.read_csv(url)

df

,outlook,temp,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


In [35]:
X = df.iloc[:, :-1].values
X

array([['sunny', 'hot', 'high', False],
       ['sunny', 'hot', 'high', True],
       ['overcast', 'hot', 'high', False],
       ['rainy', 'mild', 'high', False],
       ['rainy', 'cool', 'normal', False],
       ['rainy', 'cool', 'normal', True],
       ['overcast', 'cool', 'normal', True],
       ['sunny', 'mild', 'high', False],
       ['sunny', 'cool', 'normal', False],
       ['rainy', 'mild', 'normal', False],
       ['sunny', 'mild', 'normal', True],
       ['overcast', 'mild', 'high', True],
       ['overcast', 'hot', 'normal', False],
       ['rainy', 'mild', 'high', True]], dtype=object)

In [46]:
def countYesNo(df, feature):
    count = {}
    for i in range(len(df[feature])):
        if df[feature][i] in count:
            if df['play'][i] == 'yes':
                count[df[feature][i]]['yes'] += 1
            else:
                count[df[feature][i]]['no'] += 1
        else:
            if df['play'][i] == 'yes':
                count[df[feature][i]] = {'yes': 1, 'no': 0}
            else:
                count[df[feature][i]] = {'yes': 0, 'no': 1}
    return count
outlook = countYesNo(df, 'outlook')
temperature = countYesNo(df, 'temp')
humidity = countYesNo(df, 'humidity')
windy = countYesNo(df, 'windy')

In [47]:
outlook

{'sunny': {'yes': 2, 'no': 3},
 'overcast': {'yes': 4, 'no': 0},
 'rainy': {'yes': 3, 'no': 2}}

In [48]:
temperature

{'hot': {'yes': 2, 'no': 2},
 'mild': {'yes': 4, 'no': 2},
 'cool': {'yes': 3, 'no': 1}}

In [54]:
humidity

{'high': {'yes': 3, 'no': 4}, 'normal': {'yes': 6, 'no': 1}}

In [55]:
windy

{np.False_: {'yes': 6, 'no': 2}, np.True_: {'yes': 3, 'no': 3}}

In [57]:
def CalProbability(count, feature, value):
    return count[value][feature] / (count[value]['yes'] + count[value]['no'])
CalProbability(windy, 'yes', False)

0.75

In [53]:
def NaiveBayes(df, outlook, temperature, humidity, windy, x):
    p_yes = len(df[df['play'] == 'yes']) / len(df)
    p_no = len(df[df['play'] == 'no']) / len(df)
    p_x_yes = CalProbability(outlook, 'yes', x[0]) * CalProbability(temperature, 'yes', x[1]) * CalProbability(humidity, 'yes', x[2]) * CalProbability(windy, 'yes', x[3])
    p_x_no = CalProbability(outlook, 'no', x[0]) * CalProbability(temperature, 'no', x[1]) * CalProbability(humidity, 'no', x[2]) * CalProbability(windy, 'no', x[3])
    p_yes_x = p_x_yes * p_yes
    p_no_x = p_x_no * p_no
    return p_yes_x, p_no_x
NaiveBayes(df, outlook, temperature, humidity, windy, ['sunny', 'hot', 'normal', False])

(0.08265306122448979, 0.0038265306122448974)